In [ ]:
#https://eightportions.com/datasets/Recipes/


In [1]:
!git clone https://github.com/huggingface/transformers

fatal: destination path 'transformers' already exists and is not an empty directory.


In [2]:
!cd transformers; pip3 install .

Processing /home/jupyter/GPT2/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.16.0.dev0-py3-none-any.whl size=3414773 sha256=9b5aab8454b20eaa534673f1517636e609e08ce54660ddf6f3a0e4f197c21235
  Stored in directory: /tmp/pip-ephem-wheel-cache-nvmf96mp/wheels/39/86/5c/283acb0825b1597200d7c6e54ed3dd13d4c33e476ae8d572e5
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.16.0.dev0
    Uninstalling transformers-4.16.0.dev0:
      Successfully uninstalled transformers-4.16.0.dev0


In [3]:
from  model_demo import demo
#demo ('Ingredients: 1 cup flour \n 1 cup sugar \n cocoa \n beer \n Instructions:')

In [4]:
import matplotlib
%matplotlib inline
matplotlib.rcParams['figure.dpi'] = 200
import matplotlib.pyplot as plt
plt.style.use('ggplot')

import seaborn as sns

import pandas as pd
import json

In [5]:
!pwd

/home/jupyter/GPT2


In [6]:
data_dict = json.load(open('./recipes_raw/recipes_raw_nosource_ar.json', 'r'))

In [7]:
list(data_dict.items())[0]

('rmK12Uau.ntP510KeImX506H6Mr6jTu',
 {'title': 'Slow Cooker Chicken and Dumplings',
  'ingredients': ['4 skinless, boneless chicken breast halves ADVERTISEMENT',
   '2 tablespoons butter ADVERTISEMENT',
   '2 (10.75 ounce) cans condensed cream of chicken soup ADVERTISEMENT',
   '1 onion, finely diced ADVERTISEMENT',
   '2 (10 ounce) packages refrigerated biscuit dough, torn into pieces ADVERTISEMENT',
   'ADVERTISEMENT'],
  'instructions': 'Place the chicken, butter, soup, and onion in a slow cooker, and fill with enough water to cover.\nCover, and cook for 5 to 6 hours on High. About 30 minutes before serving, place the torn biscuit dough in the slow cooker. Cook until the dough is no longer raw in the center.\n',
  'picture_link': '55lznCYBbs2mT8BTx6BTkLhynGHzM.S'})

In [8]:
df = pd.DataFrame()

sources , titles, ingredients, instructions = [], [], [], []
for source in ['ar', 'epi', 'fn']:
    data_dict = json.load(open(f'./recipes_raw/recipes_raw_nosource_{source}.json', 'r'))
    for _, recipe in data_dict.items() :
        #print (recipe['title'] )
        if ('title' in recipe) and ('ingredients' in recipe) and ('instructions' in recipe):
            sources.append(source)
            titles.append (recipe['title'])
            ingredients.append([ ing.replace('ADVERTISEMENT', '') for ing in recipe['ingredients']])
            instructions.append ( str(recipe['instructions']).replace('ADVERTISEMENT', '').replace('\n', ' ') )
       
df['source'] =  sources 
df['title'] = titles
df['ingredients'] =  ingredients
df['instructions'] =  instructions 

In [9]:
display(df.sample(5))
print(df.shape)

,source,title,ingredients,instructions
55893,epi,Evening in Kingston,[1 1/2 cups amber rum (such as Appleton Estate...,Combine 1 1/2 cups amber rum (such as Appleton...
30022,ar,Easy Cheese Dip,"[1/2 cup cream cheese , 1/4 cup sour cream , 1...",Mix cream cheese and sour cream together in a ...
37042,ar,Maple Apple Bourbon Glaze,"[2/3 cup apple juice , 1/3 cup grade B maple s...","Combine apple juice, maple syrup, bourbon, bro..."
52711,epi,Dark Chocolate Ganache,"[2 3/4 cups mini semisweet chocolate chips, 2 ...",1 Put the chocolate chips in a large heatproof...
102870,fn,Buttermilk Southwestern Grilled Chicken,"[2 cups buttermilk, 1 lime, zested and juiced,...","Whisk together the buttermilk, lime zest and j..."


(124647, 4)


In [10]:
df['ingredient_count'] = df['ingredients'].str.len().fillna(0).astype(int)
df['instruction_length'] = df['instructions'].str.split().str.len().fillna(0).astype(int)
# sns.boxplot(data=df, x='source', y='ingredient_count')
#sns.boxplot(data=df, x='source', y='instruction_length', showfliers=False)

In [11]:
print(df.shape)
df = df[(df.ingredient_count >= 3) & (df.instruction_length >= 12) ]
print(df.shape)

(124647, 6)
(121639, 6)


In [12]:
special_token = ' <|endoftext|> '

In [13]:
df ['combined'] = ' \n Ingredients: \n ' +  df['ingredients'].str.join(' \n ') + ' \n Instructions: \n ' + df['instructions'] + special_token

In [14]:
print(df ['combined'].iloc[0] )

 
 Ingredients: 
 4 skinless, boneless chicken breast halves  
 2 tablespoons butter  
 2 (10.75 ounce) cans condensed cream of chicken soup  
 1 onion, finely diced  
 2 (10 ounce) packages refrigerated biscuit dough, torn into pieces  
  
 Instructions: 
 Place the chicken, butter, soup, and onion in a slow cooker, and fill with enough water to cover. Cover, and cook for 5 to 6 hours on High. About 30 minutes before serving, place the torn biscuit dough in the slow cooker. Cook until the dough is no longer raw in the center.  <|endoftext|> 


In [15]:
dataset_train = df[:120000].combined.values
dataset_val = df[120000:].combined.values

In [16]:
with open('data/dataset_train.txt', 'w') as f :
    f.write('\n'.join(dataset_train))
with open('data/dataset_val.txt', 'w') as f :
    f.write('\n'.join(dataset_val))

In [17]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [18]:
tokenizer = AutoTokenizer.from_pretrained('distilgpt2')
model = AutoModelForCausalLM.from_pretrained('distilgpt2')

In [19]:
prompt_text = "Today is Friday, I am going to "
encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False, return_tensors='pt')
encoded_prompt

tensor([[8888,  318, 3217,   11,  314,  716, 1016,  284,  220]])

In [20]:
out_sequences = \
model.generate (input_ids = encoded_prompt, max_length = 200, temperature = .9, top_k=20, top_p=.9, 
                repetition_penalty= 1, do_sample=True, num_return_sequences=2)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [21]:
print(tokenizer.decode(out_sequences [0]))
print(tokenizer.decode(out_sequences [1]))

Today is Friday, I am going to   open a new window to all the new   people   on   Facebook,   Twitter,   Tumblr.

I hope I can get into a few more   things.
-  My wife and I have a   car   on my   car     home in Florida  
We are in the middle of the winter when I have a   car         home in Florida  
It is a very important time to   open   a new window to all the   people   on   Facebook,   Tumblr, and    Tumblr. I would like to share with you the new   people   on   Facebook,   Tumblr, and    Tumblr.
I am  planning to open a  new  window to all the   people   on   Facebook
Today is Friday, I am going to                        
The most important thing to note, is that when you take a trip to a city where you will find many people that are very similar to you and are quite different than you.
In other words, you will find people that are very similar to you and are quite different from you and are quite different from you and are quite different from you and are quite different from 

In [22]:
!pwd

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/home/jupyter/GPT2


In [26]:
!bash run_experiments.sh

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
01/16/2022 22:17:53 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 4, distributed training: False, 16-bits training: False
01/16/2022 22:17:53 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=4,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=F

In [37]:
#model.save('./trained_models/epochs_10')

In [34]:
!pwd


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/home/jupyter/GPT2
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/home/jup

In [56]:
#prediction

model = AutoModelForCausalLM.from_pretrained('/home/jupyter/GPT2/experiments/epochs_10/')
tokenizer = AutoTokenizer.from_pretrained('distilgpt2')


In [57]:
with open('./data/dataset_val.txt', 'r') as f:
    data = f.read()


In [58]:
special_token ="<|endoftext|>"

In [59]:
data = data.split(special_token)
print(data[0])

 
 Ingredients: 
 1 1/2 pounds boneless beef sirloin 
 1 tablespoon soy sauce 
 2 tablespoons sesame oil 
 1 tablespoon rice wine or sherry 
 1 egg white, lightly beaten 
 1/2 teaspoon salt 
 2 tablespoons peanut or corn oil 
 4 dried red chiles, split 
 1 tablespoon minced garlic 
 1/2 tablespoon grated ginger 
 1 teaspoon Szechwan pepper, toasted and crushed 
 2 scallions, cut in 1/2-inch pieces 
 1 red bell pepper, cut in pieces 
 2 tablespoons soy sauce 
 3 tablespoons rice wine or sherry 
 2 tablespoons Chinese black vinegar or balsamic 
 1 teaspoon sugar 
 1 cup chicken broth 
 1 tablespoon cornstarch, dissolved in 2 tablespoons water 
 1/3 cup roasted peanuts 
 Instructions: 
 Trim fat from the steak and cut into 1-inch cubes. Combine the soy sauce, sesame oil, rice wine/sherry, egg white and salt in a glass bowl. Add the beef and stir to coat. Marinate for 1 hour, covered in the refrigerator. Place peanut/corn oil in a wok, swirling to coat the sides, and place over high heat. 

In [60]:
prompt_text = data[100].split("Instructions:")[0] + '\n Instructions: \n'
print(prompt_text)

 
 
 Ingredients: 
 1 teaspoon honey 
 1 teaspoon active dry yeast 
 2 1/4 cups flour, plus more if needed 
 1 teaspoon kosher salt 
 Extra-virgin olive oil 
 
 Instructions: 



In [61]:
encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False, return_tensors='pt')
#encoded_prompt

In [62]:
out_sequences = \
model.generate (input_ids = encoded_prompt, max_length = 700, temperature = .9, top_k=20, top_p=.9, 
                repetition_penalty= 1, do_sample=True, num_return_sequences=4)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [63]:
out_sequences

tensor([[  220,   198,   220,  ..., 19537,   220, 50256],
        [  220,   198,   220,  ..., 50256, 50256, 50256],
        [  220,   198,   220,  ..., 50256, 50256, 50256],
        [  220,   198,   220,  ..., 50256, 50256, 50256]])

In [64]:
for i, seq in enumerate(out_sequences):
    result = tokenizer.decode(seq)
    result = result[:result.index(special_token)]
    print (f'Generated Output {i} :' )
    print(result)

Generated Output 0 :
 
 
 Ingredients: 
 1 teaspoon honey 
 1 teaspoon active dry yeast 
 2 1/4 cups flour, plus more if needed 
 1 teaspoon kosher salt 
 Extra-virgin olive oil 
 
 Instructions: 
 Watch how to make this recipe. Combine the honey, yeast, and 1 1/4 cups flour in a small bowl; stir until smooth. Let sit for about 10 minutes to activate the yeast. Let sit until the yeast is dissolved and foamy, about 30 minutes. Heat a large skillet over medium-high heat. Add 1 1/2 cups olive oil; season with salt. Add the yeast mixture and cook, stirring frequently, until the mixture is bubbling and the liquid has been absorbed, about 2 minutes. Remove from the heat; stir in 1 1/2 cups flour, 1/4 teaspoon kosher salt, 1/4 teaspoon pepper, and 1/4 cup olive oil. Let sit until the dough begins to turn light brown, about 10 minutes. Divide the dough into 4 pieces; shape each into a ball. Transfer the dough to a floured work surface; roll each into a ball about 1/8-inch thick. Cut each into 

In [65]:
special_token

'<|endoftext|>'

In [68]:
ingredients = 'blueberry\n cheese\n crackers\n pasta sauce\n'
prompt_text = ' \n Ingredients: \n ' +  ingredients + ' \n Instructions: '
print (ingredients)
encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False, return_tensors='pt')
out_sequences = \
model.generate (input_ids = encoded_prompt, max_length = 700, temperature = .9, top_k=20, top_p=.9, 
                repetition_penalty= 1, do_sample=True, num_return_sequences=4)
for i, seq in enumerate(out_sequences):
    result = tokenizer.decode(seq)
    result = result[:result.index(special_token)]
    print (f'Generated Output {i} :' )
    print(result)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


blueberry
 cheese
 crackers
 pasta sauce

Generated Output 0 :
 
 Ingredients: 
 blueberry
 cheese
 crackers
 pasta sauce
 
 Instructions: 
 In a large pot of boiling salted water cook the blueberries in 2 batches until tender, about 8 minutes. Drain and cool completely. In a bowl stir together the blueberries and the cheese until well blended. Chill the mixture, covered, until it is cold. Remove the cheese from the refrigerator 1 hour before serving. 
Generated Output 1 :
 
 Ingredients: 
 blueberry
 cheese
 crackers
 pasta sauce
 
 Instructions: 
 In a large bowl, mix together the blueberries, crackers, and cornflake crumbs. Spread the mixture on top of the blueberry mixture, and refrigerate for at least 2 hours or overnight. Preheat oven to 350 degrees F. In a large skillet over medium heat, melt butter. Add the blueberries and saute until slightly browned. Remove from heat, and set aside. Add the cornflake crumb mixture to the skillet, and cook until the cornflake crumb mixture is 